# Cryosphere model Comparison tool (CmCt) Dynamic Thickness Tool

In [1]:
import numpy as np
import h5py
import os
import xarray as xr
import pyproj
import cartopy.crs as ccrs
import time
from datetime import datetime
import netCDF4
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.io.shapereader as shpreader
import concurrent.futures

## About This Tool

The CmCt Dynamic Thickness Tool compares the user's uploaded ice sheet model(s) to dynamic thickness data from IMAU, GSFC, and/or GEMB observation data. This observation data is a rectangular grid in UTM zone 24N coordinates but the model is expected as a rectangular grid in ISMIP6 polar-stereographic coordinates (see the Input Data Requirements section). The model data is bilinearly interpolated to observation space and the residual is outputted as a netCDF4 file in observation space. 

The model currently uses these three datasets made by CREDIT HERE, and available upon request from ACCESS HERE. These datasets include dynamic thickness anomaly data from balance years 1995 to 2021: `Dynamic_h_Greenland_GEMB_1994_2021.nc`, `Dynamic_h_Greenland_GSFC_1994_2021.nc`, and `Dynamic_h_Greenland_IMAU_1994_2020.nc`. See OBSERVATION DATA DOCUMENTATION HERE for more information about these datasets.

### Multiple Comparisons
This tool uses parallel processing to allow the user to efficiently compare multiple model files with multiple observation files. A comparison can occur between any observation file (of the three listed above) and any model file listed in the `model_fns` input in any year for which both have data. The user may request any amount of comparisons be done. See the "Desired Comparisons" subsection of the "Input Data Requirements" section for more information.

## Input Data Requirements

### Input netCDF file requirements
The user may include multiple netCDF4 model files, but they must contain these four variables: x, y, time, and dh. 

#### Coordinate System (x and y variables)
Coordinates are defined on a rectangular X-Y grid in the ISMIP6 standard projected polar-stereographic space. The ISMIP6 standard projection is defined [here](https://theghub.org/groups/ismip6/wiki/ISMIP6-Projections-Greenland). This means that in the model netCDF4 file, there must be variables x and y such that x has dimensions (x,) and length A and y has dimensions (y,) and length B. If multiple netCDF4 files are entered as inputs, their x and y variables are assumed to be the same, and behavior of this tool is undefined in the case that they are not. 

#### Time
Must be an array of integers, floats, or strings representing the number of years since AD 0. "1996", 1996.0, and 1996 are all acceptable entries, but no month, day, time-of-day, or timezone information should be provided. If multiple netCDF4 files are entered as inputs, their time variables may be different (i.e. one model file may include data from 1996 to 2006, another may include data from 2000 to 2010) as long as they follow these guidelines.

#### Dh variable
The CmCt Grace Mascon tool expects the uploaded model to contain dynamic thickness data (the `dh` variable) for the comparison (should be in units of meters). dh must have dimensions (time, y, x) such that dh\[i,j,k] is the change in dynamic thickness anomaly in meters from Sep 1st, time\[i] to Aug 31st, time\[i] + 1 at the position described by (x\[k], y\[j]) in ISMIP6 polar-stereographic space.


### Desired Comparisons
The filenames of all model files the user wishes to include should be listed in the variable `model_fns` as a list of strings. The input `desired_comparisons` is a list of tuples of the form `(obs_src, idx_mod, year)`. `obs_src` should be either "IMAU", "GEMB" or "GSFC" to indicate the observation dataset. `idx_mod` is an index of `model_fns` to indicate the model dataset. `year` should be a string of the form "YYYY" to indicate the year of this comparison. For each tuple in `desired_comparisons`, this tool will interpolate the model data from `model_fns[idx_mod]` in `year` to observation space and then subtract it from the `obs_src` data in `year`.

Note that, for any comparison, `year` represents the desired balance year for the comparison. If `year` = 1996, then the observation change in dynamic thickness anomaly from Sep 1st, 1996 to Aug 31st 1997 will be compared with the modelled change in dynamic thickness anomaly from Sep 1st, 1996 to Aug 31st 1997. `year` cannot occur before the start of observation data collection (1995) or after the end of observation data collection (2021 for GSFC and GEMB, 2020 for IMAU). Likewise, `year` must correspond to an entry in the time variable of the model file.

See the Inputs section for more inputs such as filenames and plotting preferences. 

## Inputs

In [2]:
# Model filenames (see "Desired Comparisons" subsection above for more info)
model_fns = ["model_files/partial_model_file.nc", "model_files/model_file.nc", "model_files/doesnt_exist.nc"]

# Desired comparisons (see "Desired Comparisons" subsection above for more info)
desired_comparisons = [("IMAU", 0, "2008"),
                       ("GEMB", 1, "2010"),
                       ("GEMB", 0, "2010"), 
                       ("IMAU", 0, "2010"), 
                       ("IMAU", 1, "2009"),
                       ("GEMB", 1, "2011"),
                       ("GEMB", 0, "2011"), 
                       ("IMAU", 0, "2011")]

# Output filename(s) in which to store residuals
# The length of output_fns must be equal to the length of desired_comparisons if save_nc = True
save_nc = True
output_fns = ["output_files/comp1.nc",
              "output_files/comp2.nc",
              "output_files/comp3.nc",
              "output_files/comp4.nc", 
              "output_files/comp5.nc",
              "output_files/comp6.nc",
              "output_files/comp7.nc",
              "output_files/comp8.nc"]

# Plotting preferences
plot = False
which_comparison_to_plot = 0   # Index in desired_comparisons of the comparison to plot
save_plot = False   # Only relevant if plot = True
plot_fn = "output_files/plot.png"   # Only relevant if plot = True and save_plot = True

## Check Inputs

In [3]:
global error
error = False

if save_nc:
    # Check that all output filenames are unique
    if len(np.unique(output_fns)) < len(output_fns):
        print("Error: At least two paths in output_fns are identical")
        error = True

    # Check that there is one filename for each comparison
    if not (len(output_fns) == len(desired_comparisons)):
        print(f"Error: {len(desired_comparisons)} comparisons requested but {len(output_fns)} output filenames provided")
        error = True

## Read in Observation Data

In [4]:
def read_obs_file(desired_comparisons):
    global x_UTM
    global y_UTM
    global crs_wkt

    # Outputs. For a more complete description, see the runProcessing function
    all_dh_obs = []      # list of 2D arrays
    comp_2_obs_idx = np.empty((len(desired_comparisons),), dtype = "int")

    obs_paths = {"IMAU": "observation_data/Dynamic_h_Greenland_IMAU_1994_2020.nc", 
                 "GEMB": "observation_data/Dynamic_h_Greenland_GEMB_1994_2021.nc",
                 "GSFC": "observation_data/Dynamic_h_Greenland_GSFC_1994_2021.nc"}

    years_needed = {"IMAU": [], "GEMB":[], "GSFC":[]}   # Key: Source, Value: years that are needed for some comparison
    comp_needing_year = {"IMAU":{}, "GEMB":{}, "GSFC":{}}   # Key: Source, Value: (Dictionary with key: year, value: index in desired_comparisons)

    # Find out which of the files need to be read for the comparisons, and which years of those files need to be read
    for i, comp in enumerate(desired_comparisons):
        obs_src, _, year = comp
        (years_needed[obs_src]).append(year)
        (comp_needing_year[obs_src]).setdefault(year, [])
        (comp_needing_year[obs_src])[year].append(i)
    for src in years_needed:      # Convert to a set to eliminate duplicates
        list = years_needed[src]
        years_needed[src] = set(list)

    set_common_vars = False
    for src in ["IMAU", "GEMB", "GSFC"]:
        if len(years_needed[src]) == 0:    # No comparison called for this file to be read, so skip it
            continue


        # Read in file data
        fn = obs_paths[src]
        with h5py.File(fn, mode='r') as f:
            if not set_common_vars:        # Ensures this is only done once, not three times
                x_UTM = f['x'][:]
                y_UTM = f['y'][:]
                spatial_ref = f['spatial_ref']
                crs_wkt = spatial_ref.attrs['crs_wkt'].decode('UTF-8')   # Contains coordinate system data
                set_common_vars = True

            time_obs = (f['time'][:]).astype("str")
            dh = f['dh'][:]    # Has shape (len(time_obs), len(y_UTM), len(x_UTM))


        # Select only years of dh data needed for some comparison; construct outputs
        for year in years_needed[src]:
            # Check desired comparison year is not outside of bounds of observation data
            year_dt = datetime.strptime(year, "%Y")
            if datetime.strptime(time_obs[0], "%Y") > year_dt:
                print(f"Error: Desired comparison year ({year}) occurs before first date of observation data ({time_obs[0]})")
                return None, None
            if datetime.strptime(time_obs[-1], "%Y") < year_dt:
                print(f"Error: Desired comparison year ({year}) occurs after last date of observation data ({time_obs[-1]})")
                return None, None

            # Find the observation dh for the year specified
            time_idx = np.argwhere(time_obs == year)[0,0]
            dh_obs = dh[time_idx, :, ]

            # Append to all_dh_obs and update comp_2_obs_idx
            all_dh_obs.append(dh_obs)
            obs_idx = len(all_dh_obs) - 1
            for i in (comp_needing_year[src])[year]:
                comp_2_obs_idx[i] = obs_idx
                 
        dh = None   # De-allocate memory
        
    return all_dh_obs, comp_2_obs_idx

## Read in Model Data

In [5]:
def read_model_file(model_fns, desired_comparisons):
    global x_mod
    global y_mod

    # Outputs. For a more complete description, see the runProcessing function
    all_dh_mod = []
    comp_2_mod_idx = np.empty((len(desired_comparisons),), dtype = "int")

    years_needed = {}        # Key: index in model_fns, Value: List of years to be read
    comp_needing_year = {}   # Key: index in model_fns: Value: (Dictionary with key: year and value: index in desired_comparisons)

    # Find out which of the files need to be read for the comparisons
    for i, comp in enumerate(desired_comparisons):
        _, idx_mod, year = comp
        years_needed.setdefault(idx_mod, [])
        (years_needed[idx_mod]).append(year)
        comp_needing_year.setdefault(idx_mod, {})
        (comp_needing_year[idx_mod]).setdefault(year, [])
        (comp_needing_year[idx_mod])[year].append(i)
    for src in years_needed:      # Convert to a set to eliminate duplicates
        list = years_needed[src]
        years_needed[src] = set(list)

    
    set_common_vars = False
    for idx_mod in years_needed:     # Iterate over model files

        # Read file data
        fn = model_fns[idx_mod]
        with h5py.File(fn , mode='r') as f:
            if not set_common_vars:
                # See "Input Data Requirements" section for more details on the format of these variables
                x_mod = f['x'][:]
                y_mod = f['y'][:]
                set_common_vars = True

            time_mod = f['time'][:]
            time_mod = np.array([str(int(t)) for t in time_mod])
            dh = f['dh'][:]


        # Select only years of dh data needed for some comparison; construct outputs
        for year in years_needed[idx_mod]:
            # Check desired comparison year is not outside of bounds of model data
            year_dt = datetime.strptime(year, "%Y")
            if datetime.strptime(time_mod[0], "%Y") > year_dt:
                print(f"Error: Desired comparison year ({year}) occurs before first date of model data ({time_mod[0]})")
                return None, None
            if datetime.strptime(time_mod[-1], "%Y") < year_dt:
                print(f"Error: Desired comparison year ({year}) occurs after last date of model data ({time_mod[-1]})")
                return None, None

            # Find the dh in the year specified
            time_idx = np.argwhere(time_mod == year)[0,0]
            try:
                dh_mod = dh[time_idx, :, :]
            except:
                print(f"Error: Could not find year {year} in model file {fn}")
                return None, None

            # Append to all_dh_mod and update comp_2_mod_idx
            all_dh_mod.append(dh_mod)
            mod_idx = len(all_dh_mod) - 1
            for i in (comp_needing_year[idx_mod])[year]:
                comp_2_mod_idx[i] = mod_idx
                
        dh = None   # De-allocate memory
            
    return all_dh_mod, comp_2_mod_idx

## Transform Observation Coordinates from UTM-24N to ISMIP6 polar-stereographic coordinates

In [6]:
def transform_obs_coords(crs_wkt, x_UTM, y_UTM):
    # See "About This Tool" section for observation coordinate system information
    
    # Make a transformer from observation coordinates to model coordinates
    crs_utm = pyproj.crs.CRS.from_wkt(crs_wkt)    # Coordinate system of observation data
    crs_ps = pyproj.crs.CRS(ccrs.Stereographic(central_latitude= 90.0, central_longitude= -45.0, false_easting= 0.0, 
                            false_northing= 0.0, true_scale_latitude= 70.0, globe=ccrs.Globe('WGS84')))    # ISMIP6 coord sys
    utm_to_ps = pyproj.Transformer.from_crs(crs_from = crs_utm, crs_to = crs_ps)

    # Format observation coordinate data into two 1D arrays of the same length
    xv, yv = np.meshgrid(x_UTM, y_UTM)
    x_UTM_points, y_UTM_points = xv.transpose().flatten(), yv.transpose().flatten()

    # Transform observation coordinate points to points in polar-stereographic space
    x_obs, y_obs = utm_to_ps.transform(x_UTM_points, y_UTM_points)
    return x_obs, y_obs

## Define Bilinear Interpolation Function

In [7]:
def bilinear_interp(x, y, arr, x_query, y_query):
    """
    Bilinearly interpolates the value of arr, defined on a rectangular grid by x and y, at the set of points defined 
    by x_query and y_query. x, y, x_query, and y_query must all be in the same coordinate system.

    Parameters:
    arr: An array of floats of shape (ny, nx)
    x, y: x has shape (nx,) and y has shape (ny,). The entry arr[i][j] corresponds to the point in space defined by 
    y[i] and x[j]. The entries of x and y must be evenly spaced
    x_query, y_query: Both must have shape (n,), where n is the number of query points. It is assumed that x, y, 
    x_query, and y_query are defined in the same coordinate system and that x and x_query represent the same dimension, 
    while y and y_query represent the other dimension 

    Returns: 
    arr_query: An array of floats of shape (n,). arr_query[i] represents the estimated value of arr at the position 
    defined by x_query[i] and y_query[i], estimated using bilinear interpolation. If any query point i is outside the 
    bounds defined by x and y, arr_query[i] will be NaN
    """

    # Query points
    x_query = np.asarray(x_query)
    y_query = np.asarray(y_query)

    # Find the spacing of the x and y arrays
    sx, sy = x[1] - x[0], y[1] - y[0]
    a = sx * sy  # area of each grid cell

    # Find the indices of the four closest points in arr to each query point
    x0 = np.floor((x_query - x[0]) / sx).astype(int)   # Left
    x1 = x0 + 1                                        # Right
    y0 = np.floor((y_query - y[0]) / sy).astype(int)   # Bottom
    y1 = y0 + 1                                        # Top

    # ma[i] is True if both x_query[i] and y_query[i] are within the bounds of the grid defined by x and y
    ma = (x0 >= 0) & (x1 < len(x)) & (y0 >= 0) & (y1 < len(y))

    # Values at each of the reference coordinates
    Ia, Ib, Ic, Id = (np.empty_like(x_query, dtype=np.float64), np.empty_like(x_query, dtype=np.float64),
                     np.empty_like(x_query, dtype=np.float64), np.empty_like(x_query, dtype=np.float64))  # Allocate memory
    Ia[~ma], Ib[~ma], Ic[~ma], Id[~ma] = np.nan, np.nan, np.nan, np.nan    # Set to NaN where x or y are out of bounds
    Ia[ma] = arr[ y0[ma], x0[ma] ]    # Bottom left
    Ib[ma] = arr[ y1[ma], x0[ma] ]    # Top left
    Ic[ma] = arr[ y0[ma], x1[ma] ]    # Bottom right
    Id[ma] = arr[ y1[ma], x1[ma] ]    # Top right

    # Weights for each reference coordinate
    wa, wb, wc, wd = (np.empty_like(x_query, dtype=np.float64), np.empty_like(x_query, dtype=np.float64),
                     np.empty_like(x_query, dtype=np.float64), np.empty_like(x_query, dtype=np.float64))  # Allocate memory
    wa[~ma], wb[~ma], wc[~ma], wd[~ma] = np.nan, np.nan, np.nan, np.nan    # Set to NaN where x or y are out of bounds
    wa[ma] = ((x[x1[ma]] - x_query[ma]) * (y[y1[ma]] - y_query[ma])) / a
    wb[ma] = ((x[x1[ma]] - x_query[ma]) * (y_query[ma] - y[y0[ma]])) / a
    wc[ma] = ((x_query[ma] - x[x0[ma]]) * (y[y1[ma]] - y_query[ma])) / a
    wd[ma] = ((x_query[ma] - x[x0[ma]]) * (y_query[ma] - y[y0[ma]])) / a

    return wa*Ia + wb*Ib + wc*Ic + wd*Id     # arr_query in documentation

## Interpolate Model to Observation Space

In [8]:
def interp_one_model_to_obs_space(dh_mod):
    """
    Interpolates a model dh 2D array (representing a single balance year) to observation space and 
    reshape to newshape
    """
    
    # Bilinearly interpolate model to observation space
    model_dh_interped_flat = bilinear_interp(x_mod, y_mod, dh_mod, x_query, y_query)

    # Reshape model array to the same shape as the observation array
    dh_mod_interped = np.empty_like(I_, dtype = np.float64)
    dh_mod_interped[~I_] = np.nan     # Fill with NaN at indices where dh_obs is NaN
    dh_mod_interped[I_] = model_dh_interped_flat
    dh_mod_interped = np.reshape(dh_mod_interped, newshape, order = "F")
    return dh_mod_interped

In [9]:
def interpolate_all_models_to_observation_space(x_obs, y_obs, dh_obs, all_dh_mod):
    """
    Wrapper function for interp_one_model_to_obs_space which parallelizes the interpolation operation of all models
    """
    
    # Set inputs to interp_one_model_to_obs_space that will not vary between calls
    global I_
    global x_query, y_query     # Interpolation query points (does not include points which are NaN in observation data)
    global newshape             # Shape that the model will be interpolated to
    
    I_ = (~np.isnan(dh_obs)).transpose().flatten()   # I_[i] is False if x_obs[i], y_obs[i] correspond to a point where dh_obs is NaN, True otherwise
    x_query, y_query = x_obs[I_], y_obs[I_]
    newshape = dh_obs.shape

    # Parallelize the interpolation and reshaping step in order to speed up
    if len(all_dh_mod) == 1:
        return [interp_one_model_to_obs_space(all_dh_mod[0])]
    else:
        all_dh_mod_interped = []
        inputs = []
        with concurrent.futures.ProcessPoolExecutor() as executor:
            generator = executor.map(interp_one_model_to_obs_space, all_dh_mod)
        for dh_mod_interped in generator:
            all_dh_mod_interped.append(dh_mod_interped)
        return all_dh_mod_interped

## Subtract Model and Residual

In [10]:
def compute_residuals(all_dh_obs, comp_2_obs_idx, all_dh_mod_interped, comp_2_mod_idx):
    """
    For each comparison in desired_comparisons, computes the residual and outputs all residuals
    as a list such that all_dh_res[i] corresponds to desired_comparison[i]

    See runProcessing function for details on the inputs to this function
    """
    all_dh_res = []
    for i in range(len(comp_2_obs_idx)):
        obs_idx = comp_2_obs_idx[i]
        mod_idx = comp_2_mod_idx[i]
        all_dh_res.append(all_dh_mod_interped[mod_idx] - all_dh_obs[obs_idx])  # residual = model - observation
    return all_dh_res

## Plot Observation, Model, and Residual

In [11]:
def plot_obs_mod_res(extent, dh_obs, dh_mod_interped, dh_res, year, save_plot):
    mpl.rcParams.update(mpl.rcParamsDefault)
    mpl.rcParams['figure.dpi'] = 100
    fig = plt.figure(figsize=(24,14))

    """
    crs = ccrs.UTM('24')
    coords = {"x": x_UTM, "y": y_UTM}
    dims = ["y", "x"]
    
    # Observed
    ax1 = plt.subplot(projection = crs)
    #ax1 = plt.subplot(131)
    #ax1.set_extent(extent) # Map bounds, [west, east, south, north]
    xr_obs = xr.DataArray(dh_obs, coords = coords, dims = dims)
    xr.plot.imshow(xr_obs, ax=ax1, transform = crs) 
    ax1.set_title("dh_obs")
    """
    mins = [np.min(dh_obs), np.min(dh_mod_interped), np.min(dh_res)]
    min = np.min(mins)
    maxs = [np.max(dh_obs), np.max(dh_mod_interped), np.max(dh_res)]
    max = np.max(maxs)
    

    ax1 = plt.subplot(131)
    im1 = ax1.imshow(dh_obs, aspect = "equal", origin = "lower", extent = extent, vmin = min, vmax = max, cmap = "RdYlBu")
    ax1.set_xbound(lower = extent[0], upper = extent[1])
    ax1.set_ybound(lower = extent[2], upper = extent[3])
    ax1.set_xlim(left = extent[0], right = extent[1])
    ax1.set_ylim(bottom = extent[2], top = extent[3])
    ax1.set_title(f"Observed Dynamic Thickness for {year}", fontsize = 18)
    
    
    # Model
    ax2 = plt.subplot(132)
    im2 = ax2.imshow(dh_mod_interped, aspect = "equal", origin = "lower", extent = extent, vmin = min, vmax = max, cmap = "RdYlBu")
    ax2.set_xbound(lower = extent[0], upper = extent[1])
    ax2.set_ybound(lower = extent[2], upper = extent[3])
    ax2.set_xlim(left = extent[0], right = extent[1])
    ax2.set_ylim(bottom = extent[2], top = extent[3])
    ax2.set_title(f"Modelled Dynamic Thickness for {year} (interpolated)", fontsize = 18)
    

    # Residual
    ax3 = plt.subplot(133)
    im3 = ax3.imshow(dh_res, aspect = "equal", origin = "lower", extent = extent, vmin = min, vmax = max, cmap = "RdYlBu")
    ax3.set_xbound(lower = extent[0], upper = extent[1])
    ax3.set_ybound(lower = extent[2], upper = extent[3])
    ax3.set_xlim(left = extent[0], right = extent[1])
    ax3.set_ylim(bottom = extent[2], top = extent[3])
    ax3.set_title(f"Residual Dynamic Thickness for {year}", fontsize = 18)
    
    
    fig.subplots_adjust(right=0.85)
    cbar_ax = fig.add_axes([0.875, 0.15, 0.02, 0.7])
    fig.colorbar(im1, cax=cbar_ax)
    fig.colorbar(im2, cax=cbar_ax)
    fig.colorbar(im3, cax=cbar_ax)
    cbar_ax.set_ylabel("meters", fontsize = 16)

    fig.suptitle("Dynamic Thickness Anomaly Change", fontsize = 24)
    
    if save_plot:
        plt.savefig(plot_fn)
    plt.show()

## Save to netCDF File


In [12]:
def save_one_residual_to_netcdf(tuple):
    """
    Saves the residual of a single comparison to a netcdf file
    """
    fn, dh_res, year = tuple
    
    rootgrp = netCDF4.Dataset(fn, "w", format="NETCDF4")
    
    # Record coordinate system
    spatial_ref_var = rootgrp.createVariable("spatial_ref", "i8")
    spatial_ref_var[:] = 0
    spatial_ref_var.crs_wkt = crs_wkt
    
    # Set up x and y variables
    x_dim = rootgrp.createDimension("x", len(x_UTM))
    y_dim = rootgrp.createDimension("y", len(y_UTM))
    x_var = rootgrp.createVariable("x", "f4", ("x",))
    y_var = rootgrp.createVariable("y", "f4", ("y",))
    x_var[:], y_var[:] = x_UTM, y_UTM
    x_var.units, y_var.units = "meter", "meter"
    
    # Set up dynamic thickness residual variable
    dh_res_var = rootgrp.createVariable("dh_res", "f4", ("y","x",))
    dh_res_var[:,:] = dh_res
    dh_res_var.coordinates = "spatial_ref"
    dh_res_var.grid_mapping = "spatial_ref"
    dh_res_var.long_name = "Residual of annual dynamic mass/ice thickness (assume ice density of 917 kg/m3) change between Sep 1 of " + year + " and " + str(int(year) + 1)
    dh_res_var.units = "meters"
    rootgrp.close()

In [13]:
def save_residuals_to_netcdf(output_fns, all_dh_res, x_UTM, y_UTM, desired_comparisons, crs_wkt):
    """
    Wrapper function for save_one_residual_to_netcdf which parallelizes the saving of all residuals
    """
    
    if len(all_dh_res) == 1:
        save_one_residual_to_netcdf(output_fns[0], all_dh_res[0], (desired_comparisons[0])[2])
    else:
        inputs = []
        for i in range(len(desired_comparisons)):
            inputs.append((output_fns[i], all_dh_res[i], (desired_comparisons[i])[2]))
        
        with concurrent.futures.ProcessPoolExecutor() as executor:
            results = executor.map(save_one_residual_to_netcdf, inputs)
            for result in results:
                pass

## Compute Comparison

In [14]:
def runProcessing():
    """
    Before reading, see the "Desired Comparisons" subsection of the "Input Data Requirements" section.

    Here is a summary of what the tool does at a basic level. First it reads in the model and 
    observation data for the appropriate years. It then transforms the spatial coordinates of the 
    observation data from a grid in UTM-24N to a set of non-gridded points in polar-stereographic
    coordinates. Then, using the fact that the model data is a grid in polar-stereographic, the tool
    uses bilinear interpolation to estimate the value of the model data at the locations of the 
    observation data. This and the observation data are then subtracted, giving the residual. Below 
    is a more detailed account of the pipeline, which defines the behavior of important variables.

    Pipeline:
    1) Read in observation data as a list of 2D arrays called all_dh_obs. Each entry of all_dh_obs has
       the same shape, let this be (ny, nx). Each entry of all_dh_obs is for a different year or source 
       necessary in some comparison in desired_comparisons. The variable comp_2_obs_idx records which 
       comparisons need which entry of all_dh_obs: comp_2_obs_idx[i] is the index of all_dh_obs which 
       holds the observation data needed for desired_comparison[i]. The spatial information for all 
       observation data arrays is stored in x_UTM and y_UTM, and x_UTM has length nx and y_UTM has 
       length ny.
    2) Read in model data as a list of 2D arrays called all_dh_mod. Each entry of all_dh_mod is assumed 
       to have the same shape. Each entry of all_dh_mod is for a different year or model necessary in some 
       comparison in desired_comparisons. The variable comp_2_mod_idx records which comparisons need 
       which entry of all_dh_mod: comp_2_mod_idx[i] is the index of all_dh_mod which holds the model 
       data needed for desired_comparison[i]. The spatial information for all model data arrays is stored 
       in x_mod and y_mod. See the "Coordinate System" subsection of the "Input Data Requirements" section 
       for more information.
    3) Reshape coordinate information encoded in x_UTM and y_UTM to a set of points described by two 1D 
       arrays (both with length nx * ny), which are then converted into polar stereographic coordinates. 
       These arrays are x_obs and y_obs.
    4) For each entry of all_dh_mod, bilinearly interpolate from the model's grid in polar stereographic, 
       using x_obs and y_obs as the query points. This will return an estimated value of the modelled 
       value at each of the points for which the observation value is known. In other words, the model has 
       been brought to observation space. Note that this step is done in parallel.
    5) For each desired comparison, compute the residual (model - observation) in observation space
    6) (Optional) Write all residuals to netCDF4 files
    7) (Optional) For a single comparison, plot the observation, model, and residual and optionally save 
        the plot
    """
    if error:
        return None
    
    update_progress(0.0, "Starting...              ", 0.0)
    t = time.time()
    all_dh_obs, comp_2_obs_idx = read_obs_file(desired_comparisons)     # Sets global variables x_UTM, y_UTM, crs_wkt
    if all_dh_obs is None:
        return None
    update_progress(0.2, "Read Observation Data    ", time.time() - t)

    
    t = time.time()
    all_dh_mod, comp_2_mod_idx = read_model_file(model_fns, desired_comparisons)     # Sets global variables x_mod, y_mod
    if all_dh_mod is None:
        return None
    update_progress(0.4, "Read Model Data          ", time.time() - t)

    
    t = time.time()
    x_obs, y_obs,  = transform_obs_coords(crs_wkt, x_UTM, y_UTM)
    update_progress(0.5, "Transformed Coordinates  ", time.time() - t)

    
    t = time.time()
    all_dh_mod_interped = interpolate_all_models_to_observation_space(x_obs, y_obs, all_dh_obs[0], all_dh_mod)
    all_dh_mod = None    # Deallocate memory
    update_progress(0.7, "Interpolated Model       ", time.time() - t)

    t = time.time()
    all_dh_res = compute_residuals(all_dh_obs, comp_2_obs_idx, all_dh_mod_interped, comp_2_mod_idx)
    update_progress(0.8, "Computed Residuals       ", time.time() - t)
    

    if save_nc:
        t = time.time()
        save_residuals_to_netcdf(output_fns, all_dh_res, x_UTM, y_UTM, desired_comparisons, crs_wkt)
        update_progress(0.9, "Saved residuals to NetCDF", time.time() - t)
    

    if plot:
        print("Plotting....")
        t = time.time()
        i = which_comparison_to_plot
        plot_obs_mod_res((x_UTM[0], x_UTM[-1], y_UTM[0], y_UTM[-1]), all_dh_obs[comp_2_obs_idx[i]], 
                         all_dh_mod_interped[comp_2_mod_idx[i]], all_dh_res[i], (desired_comparisons[i])[2], save_plot)
        update_progress(1.0, "Plotted Results          ", time.time() - t)
    
    print("Success")
    return "Success"

In [15]:
# update processing progress bar
def update_progress(progress, title, time_elapsed_print):
    bar_length = 20
    block = int(20.0*progress)
    text = title+" [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100) + f" ({time_elapsed_print} seconds)"
    print(text)   

In [16]:
t_tot = time.time()
runProcessing()
print("Total time: " + str(time.time() - t_tot) + " seconds")

Starting...               [--------------------] 0.0% (0.0 seconds)
Read Observation Data     [####----------------] 20.0% (4.601094961166382 seconds)
Read Model Data           [########------------] 40.0% (1.0195326805114746 seconds)
Transformed Coordinates   [##########----------] 50.0% (1.405395269393921 seconds)
Interpolated Model        [##############------] 70.0% (2.9890050888061523 seconds)
Computed Residuals        [################----] 80.0% (0.21115922927856445 seconds)
Saved residuals to NetCDF [##################--] 90.0% (1.3754267692565918 seconds)
Success
Total time: 11.695570468902588 seconds
